In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Duchenne_Muscular_Dystrophy"
cohort = "GSE13608"

# Input paths
in_trait_dir = "../../input/GEO/Duchenne_Muscular_Dystrophy"
in_cohort_dir = "../../input/GEO/Duchenne_Muscular_Dystrophy/GSE13608"

# Output paths
out_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/GSE13608.csv"
out_gene_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE13608.csv"
out_clinical_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/clinical_data/GSE13608.csv"
json_path = "../../output/preprocess/Duchenne_Muscular_Dystrophy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Skeletal muscle biopsies of patients with myotonic dystrophy (DM) and non-DM neuro-muscular disorders"
!Series_summary	"Skeletal muscle biopsies from DM1, DM2, idiopathic DM (DMx), and non-DM NMD patients were compared to those from normal individuals, with focus on MEF2 and MEF2-related genes."
!Series_summary	""
!Series_summary	"Keywords: 7 diseases and 2 normal (fetal and adult) groups"
!Series_overall_design	"Skeletal muscle biopsies from 10 DM1 biopsies, 20 DM2 biopsies, 16 DMx (DM-like, no DMPK or ZNF9 expansion), 5 BMD (Becker Muscular Dystrophy), 1 MC-AD (Myotonia Congenita-Autosomal Dominant), 3 DMD (Duchenne Muscular Dystrophy), 4 TMD (Tibial Muscular Dystrophy), 3 Normal Fetal and 6 Normal individuals"
Sample Characteristics Dictionary:
{0: ['biopsy site: Biceps brachii', 'biopsy site: Deltoid', 'biopsy site: unknown', 'biopsy site: Quadriceps', 'biopsy site: Vastus lateralis', 'biopsy site: Tibialis anterior', 'biopsy site: Gastrocnemi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset includes genomic expression profiles from muscle biopsies
is_gene_available = True  # This dataset likely contains gene expression data for multiple muscle disorders

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait row identification: Row 1 contains disease information
trait_row = 1  # "DM1 patient", "DMD patient", etc.

# Age row identification: Row 2 contains age information
age_row = 2  # "age unknown", "age 55", etc.

# Gender row identification: Row 3 contains gender information
gender_row = 3  # "Gender: F", "Gender: M", etc.

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """Convert trait value to binary (0 for non-DMD, 1 for DMD)."""
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if "duchenne muscular dystrophy" in value or "dmd patient" in value:
        return 1
    else:
        return 0

def convert_age(value: str) -> Optional[float]:
    """Extract age as a continuous value."""
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if "age unknown" in value:
        return None
    elif "age" in value:
        try:
            # Extract number after "age" keyword
            age_str = value.split("age")[1].strip()
            return float(age_str)
        except (ValueError, IndexError):
            return None
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if "gender: f" in value:
        return 0
    elif "gender: m" in value:
        return 1
    elif "gender m/f pool" in value:
        return None  # Mixed samples can't be assigned to a specific gender
    return None

# Helper function used by geo_select_clinical_features
def get_feature_data(df, row_idx, feature_name, convert_func):
    """Extract and convert feature data from a row in the DataFrame."""
    if row_idx is None:
        return pd.DataFrame({feature_name: []})
    
    values = df[row_idx].tolist()
    converted_values = [convert_func(val) for val in values]
    return pd.DataFrame({feature_name: converted_values})

# 3. Save Metadata - Initial Filtering
is_trait_available = (trait_row is not None)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create clinical_data from the sample characteristics dictionary provided
    # Sample characteristics dictionary from previous step output
    char_dict = {
        0: ['biopsy site: Biceps brachii', 'biopsy site: Deltoid', 'biopsy site: unknown', 'biopsy site: Quadriceps', 
            'biopsy site: Vastus lateralis', 'biopsy site: Tibialis anterior', 'biopsy site: Gastrocnemius', 
            'biopsy site: Triceps brachii', 'biopsy site: Extensor hallucis/Digitorum longus'],
        1: ['DM1 patient', 'DM2 patient', 'DMX patient', 'Normal adult patient', 'Normal fetus patient', 
            'Baker Myotonic Dystrophy patient', 'Duchenne Muscular Dystrophy patient', 
            'Tibial Muscular Dystropy patient', 'Myotonia Congenita Autosomal Dominant patient'],
        2: ['age unknown', 'age 55', 'age 54', 'age 25', 'age 29', 'age 21', 'age 71', 'age 39', 'age 69', 
            'age 68', 'age 32', 'age 47', 'age 57', 'age 43', 'age 37', 'age 65', 'age 42', 'age 50', 
            'age 51', 'age 58', 'age 28', 'age 49', 'age 75', 'age 73', 'age 53', 'age 36', 'age 46', 
            'age 48', 'age 61', 'age 85'],
        3: ['Gender: F', 'Gender: M', 'Gender M/F pool']
    }
    
    # Convert dictionary to proper format for geo_select_clinical_features
    clinical_data = pd.DataFrame.from_dict(char_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the DataFrame
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0, nan, 0.0], 1: [0.0, 55.0, 1.0], 2: [0.0, 54.0, nan], 3: [0.0, 25.0, nan], 4: [0.0, 29.0, nan], 5: [0.0, 21.0, nan], 6: [1.0, 71.0, nan], 7: [0.0, 39.0, nan], 8: [0.0, 69.0, nan], 9: [nan, 68.0, nan], 10: [nan, 32.0, nan], 11: [nan, 47.0, nan], 12: [nan, 57.0, nan], 13: [nan, 43.0, nan], 14: [nan, 37.0, nan], 15: [nan, 65.0, nan], 16: [nan, 42.0, nan], 17: [nan, 50.0, nan], 18: [nan, 51.0, nan], 19: [nan, 58.0, nan], 20: [nan, 28.0, nan], 21: [nan, 49.0, nan], 22: [nan, 75.0, nan], 23: [nan, 73.0, nan], 24: [nan, 53.0, nan], 25: [nan, 36.0, nan], 26: [nan, 46.0, nan], 27: [nan, 48.0, nan], 28: [nan, 61.0, nan], 29: [nan, 85.0, nan]}
Clinical data saved to ../../output/preprocess/Duchenne_Muscular_Dystrophy/clinical_data/GSE13608.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 63
Header line: "ID_REF"	"GSM343029"	"GSM343030"	"GSM343031"	"GSM343032"	"GSM343033"	"GSM343034"	"GSM343035"	"GSM343036"	"GSM343037"	"GSM343038"	"GSM343039"	"GSM343040"	"GSM343041"	"GSM343042"	"GSM343043"	"GSM343044"	"GSM343045"	"GSM343046"	"GSM343047"	"GSM343048"	"GSM343049"	"GSM343050"	"GSM343051"	"GSM343052"	"GSM343053"	"GSM343054"	"GSM343055"	"GSM343056"	"GSM343057"	"GSM343058"	"GSM343059"	"GSM343060"	"GSM343061"	"GSM343062"	"GSM343063"	"GSM343064"	"GSM343065"	"GSM343066"	"GSM343067"	"GSM343068"	"GSM343069"	"GSM343070"	"GSM343071"	"GSM343072"	"GSM343073"	"GSM343074"	"GSM343075"	"GSM343076"	"GSM343077"	"GSM343078"	"GSM343079"	"GSM343080"	"GSM343081"	"GSM343082"	"GSM343083"	"GSM343084"	"GSM343085"	"GSM343086"	"GSM343087"	"GSM343088"	"GSM343089"	"GSM343090"	"GSM343091"	"GSM343092"	"GSM343093"	"GSM343094"	"GSM343095"	"GSM343096"
First data line: "1007_s_at"	483	446.71	471.33	554.88	265.14	537.91	305.4	400.95	387.56	434.66	432.39	282.27	385.39	503.47	410.8	318.

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene IDs from the snapshot, these appear to be probe IDs from an Affymetrix microarray
# (like "1007_s_at", "1053_at", etc.). These are not standard human gene symbols.
# They need to be mapped to gene symbols for better interpretation and analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain probe IDs and gene symbols
# From the preview, we can see:
# - 'ID' column contains probe IDs like "1007_s_at" which match the gene identifiers in the expression data
# - 'Gene Symbol' column contains human gene symbols like "DDR1 /// MIR4640"

# 2. Get gene mapping dataframe by extracting the ID and Gene Symbol columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Preview the mapping data
print("Gene mapping preview:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the resulting gene expression data
print("\nGene expression data preview (after mapping):")
print(f"Number of genes: {len(gene_data)}")
print(f"Number of samples: {gene_data.shape[1]}")
print(preview_df(gene_data))

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}



Gene expression data preview (after mapping):
Number of genes: 21278
Number of samples: 68
{'GSM343029': [125.24, 115.75, 93.97999999999999, 2135.54, 156.68], 'GSM343030': [109.31, 168.38, 134.31, 1467.21, 67.45], 'GSM343031': [113.71, 174.9, 119.44, 1383.6599999999999, 93.9], 'GSM343032': [107.14, 124.05, 105.66999999999999, 1574.3000000000002, 140.36], 'GSM343033': [102.77, 151.74, 117.64, 1700.99, 86.04], 'GSM343034': [100.74, 150.76, 111.8, 1631.14, 146.31], 'GSM343035': [99.52, 140.66, 105.67, 1924.62, 166.86], 'GSM343036': [110.14, 186.51, 114.62, 1101.58, 77.69], 'GSM343037': [119.63, 141.71, 149.63, 1820.77, 140.92], 'GSM343038': [113.36, 160.39, 153.93, 1790.31, 116.04], 'GSM343039': [94.31, 153.6, 108.09, 1621.31, 98.17], 'GSM343040': [108.13, 160.95, 114.18, 1736.69, 74.1], 'GSM343041': [116.89, 170.59, 105.55, 1594.04, 100.08], 'GSM343042': [111.56, 129.37, 100.34, 1380.74, 62.45], 'GSM343043': [103.87, 169.32, 98.57, 1625.41, 83.63], 'GSM343044': [125.91, 163.58, 125.8000


Gene expression data saved to ../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE13608.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data from the previously saved file
try:
    # Load the previously saved clinical data
    clinical_features = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical features from {out_clinical_data_file}")
    print(f"Clinical features shape: {clinical_features.shape}")
    
    # Determine the actual column names in the clinical features dataframe
    trait_column = "0"  # Based on previous step output
    age_column = "1" if "1" in clinical_features.columns else None
    gender_column = "2" if "2" in clinical_features.columns else None
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first 5 columns):")
    if not linked_data.empty:
        print(linked_data.iloc[:5, :5])
    
    # 4. Handle missing values
    print("Missing values before handling:")
    print(f"  Trait (column {trait_column}) missing: {linked_data[trait_column].isna().sum()} out of {len(linked_data)}")
    if age_column and age_column in linked_data.columns:
        print(f"  Age (column {age_column}) missing: {linked_data[age_column].isna().sum()} out of {len(linked_data)}")
    if gender_column and gender_column in linked_data.columns:
        print(f"  Gender (column {gender_column}) missing: {linked_data[gender_column].isna().sum()} out of {len(linked_data)}")
    
    gene_cols = [col for col in linked_data.columns if col not in [trait_column, age_column, gender_column]]
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
    
    cleaned_data = handle_missing_values(linked_data, trait_column)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")
    
    # 5. Evaluate bias in trait and demographic features
    is_trait_biased = False
    if len(cleaned_data) > 0:
        trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait_column)
        is_trait_biased = trait_biased
    else:
        print("No data remains after handling missing values.")
        is_trait_biased = True
    
    # 6. Final validation and save
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=cleaned_data,
        note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs non-DMD samples."
    )
    
    # 7. Save if usable
    if is_usable and len(cleaned_data) > 0:
        # Rename columns to more descriptive names before saving
        column_mapping = {}
        if trait_column in cleaned_data.columns:
            column_mapping[trait_column] = trait
        if age_column in cleaned_data.columns:
            column_mapping[age_column] = "Age"
        if gender_column in cleaned_data.columns:
            column_mapping[gender_column] = "Gender"
            
        if column_mapping:
            cleaned_data = cleaned_data.rename(columns=column_mapping)
            
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        cleaned_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data was determined to be unusable or empty and was not saved")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # Handle the error case by still recording cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Mark as not available due to processing issues
        is_biased=True, 
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Error processing data: {str(e)}"
    )
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (19845, 68)
First few genes with their expression values after normalization:
          GSM343029  GSM343030  GSM343031  GSM343032  GSM343033  GSM343034  \
Gene                                                                         
A1BG         125.24     109.31     113.71     107.14     102.77     100.74   
A1BG-AS1     115.75     168.38     174.90     124.05     151.74     150.76   
A1CF          93.98     134.31     119.44     105.67     117.64     111.80   
A2M         2135.54    1467.21    1383.66    1574.30    1700.99    1631.14   
A2M-AS1      156.68      67.45      93.90     140.36      86.04     146.31   

          GSM343035  GSM343036  GSM343037  GSM343038  ...  GSM343087  \
Gene                                                  ...              
A1BG          99.52     110.14     119.63     113.36  ...     104.37   
A1BG-AS1     140.66     186.51     141.71     160.39  ...     144.49   
A1CF         105.67     114.62     149.63     153.93  ...  

Normalized gene data saved to ../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE13608.csv
Loaded clinical features from ../../output/preprocess/Duchenne_Muscular_Dystrophy/clinical_data/GSE13608.csv
Clinical features shape: (3, 30)
Linked data shape: (98, 19848)
Linked data preview (first 5 rows, first 5 columns):
     0     1    2  A1BG  A1BG-AS1
0  0.0   NaN  0.0   NaN       NaN
1  0.0  55.0  1.0   NaN       NaN
2  0.0  54.0  NaN   NaN       NaN
3  0.0  25.0  NaN   NaN       NaN
4  0.0  29.0  NaN   NaN       NaN
Missing values before handling:
Error processing data: '0'
Abnormality detected in the cohort: GSE13608. Preprocessing failed.
Data was determined to be unusable and was not saved
